In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

##############################################
############### gaussianBlur #################
##############################################
def gaussianBlur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

##############################################
############### calibrateCamera ##############
##############################################
def calibrateCamera(nx, ny):
    objpoints = [] # 3D points in real world space
    imgpoints = [] # 2D points in image plane
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0), .... , (8,5,0)
    objp = np.zeros((nx*ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2) # x,y co-ordinates
    
    for image_path in glob.glob("camera_cal/*.jpg"):
        img = cv2.imread(image_path)
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        # If found, draw corners
        if ret == True:
            imgpoints.append(corners)
            objpoints.append(objp)
            #corner_img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
            #plt.imshow(corner_img)
            #plt.show()
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None) 
    return ret, mtx, dist

#########################################################
############### getUndistortImage #######################
#########################################################
def getUndistortImage(img, mtx, dist):
    undistorted_image = cv2.undistort(img, mtx, dist, None, mtx)
    
    # Return undistorted image
    return undistorted_image

#########################################################
############### applyGradientXThresholds ################
#########################################################
def applyGradientXThresholds(img, sobel_kernel=3, mag_thresh=(0,255)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobelx = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    sxbinary = np.zeros_like(scaled_sobelx)
    sxbinary[(scaled_sobelx >= mag_thresh[0]) & (scaled_sobelx <= mag_thresh[1])] = 1
    
    return sxbinary

#########################################################
############### applyGradientYThresholds ################
#########################################################
def applyGradientYThresholds(img, sobel_kernel=3, mag_thresh=(0,255)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    abs_sobely = np.absolute(sobely) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobely = np.uint8(255*abs_sobely/np.max(abs_sobely))
    
    sybinary = np.zeros_like(scaled_sobely)
    sybinary[(scaled_sobely >= mag_thresh[0]) & (scaled_sobely <= mag_thresh[1])] = 1
    
    return sybinary

#########################################################
############### applyGradientThresholds #################
########################################################
def applyGradientThresholds(img, sobel_kernel=3, mag_thresh=(0,255), dir_thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    
    # Take the absolute value of the gradient direction,
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[((gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])) | ((absgraddir >= dir_thresh[0]) & (absgraddir <= dir_thresh[1]))] = 1

    # Return the binary image
    return binary_output

########################################################
############### applyGradientDirThresholds #############
########################################################
def applyGradientDirThresholds(img, sobelx, sobely, dir_thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Take the absolute value of the gradient direction,
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(absgraddir)
    binary_output[((absgraddir >= dir_thresh[0]) & (absgraddir <= dir_thresh[1]))] = 1

    # Return the binary image
    return binary_output

#########################################################
############### applyHLSColorThresholds #################
#########################################################
def applyHLSColorThresholds(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    binary_output = np.zeros_like(s_channel)
    binary_output[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    return binary_output


#########################################################
############### getPerspectiveTransformMatrix ###########
#########################################################
def getPerspectiveTransformMatrix(img):
    src = np.float32([(210, 710), (520, 500), (770, 500), (1100, 710)])
    dst = np.float32([(210, 710), (210, 500), (1100, 500), (1100, 710)])
    
    #pts = np.array([[210, 710],[530, 500],[760, 500],[1100, 710]], np.int32)
    #pts = pts.reshape((-1,1,2))
    #cv2.polylines(img,[pts],True,(0,255,255))
    #plt.show()
    
    # Given src and dst points, calculate the perspective transform matrix
    Mdirect = cv2.getPerspectiveTransform(src, dst)
    Minverse = cv2.getPerspectiveTransform(dst, src)
    return Mdirect, Minverse

#########################################################
###################### getHistogram #####################
#########################################################
def getHistogram(img):
    # Grab only the bottom half of the image
    # Lane lines are likely to be mostly vertical nearest to the car
    bottom_half = img[img.shape[0]//2:,:]

    # Sum across image pixels vertically - make sure to set an `axis`
    # i.e. the highest areas of vertical lines should be larger values
    histogram = np.sum(bottom_half, axis=0)
    
    return histogram

#########################################################
###################### find_lane_pixels #################
#########################################################
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img


#########################################################
###################### fit_polynomial ###################
#########################################################
def fit_polynomial(binary_warped):
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    # Fit a second order polynomial to each using `np.polyfit`
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    return left_fitx, right_fitx, ploty

#########################################################
###################### render_lane ######################
#########################################################
def render_lane(image, Minverse, ploty, left_fitx, right_fitx):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(image[:,:,0]).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Draw area between the lane lines with green color
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minverse, (image.shape[1], image.shape[0])) 

    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 1, 0)
    
    return result 

In [9]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from moviepy.editor import VideoFileClip
from IPython.display import HTML

global mtx
global dist

#########################################################
###################### plot_images ######################
#########################################################
def plot_images(original_image, output_image):
    # Plot the result
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(original_image)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(output_image, cmap='gray')
    ax2.set_title('Output Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()

#########################################################
###################### write_output_image ###############
#########################################################
def write_output_image(image_name, output_image):
    cv2.imwrite("output_images/Output_" + image_name, output_image)

#########################################################
###################### process_image ####################
#########################################################
def process_image(img):
    global mtx
    global dist
    undistorted_img = getUndistortImage(img, mtx, dist)

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 5
    blur_gray = gaussianBlur(undistorted_img, kernel_size)

    sobel_binary_image = applyGradientXThresholds(blur_gray, sobel_kernel=5, mag_thresh=(30,100))

    hls_binary_output = applyHLSColorThresholds(blur_gray, thresh=(130, 255) )

    main_binary_image = cv2.bitwise_or(sobel_binary_image, hls_binary_output)

    Mdirect, Minverse = getPerspectiveTransformMatrix(img)

    binary_img_size = (main_binary_image.shape[1], main_binary_image.shape[0])

    warped_binary_image = cv2.warpPerspective(main_binary_image, Mdirect, binary_img_size, flags=cv2.INTER_LINEAR)

    main_img_size = (img.shape[1], img.shape[0])

    warped_img = cv2.warpPerspective(img, Mdirect, main_img_size, flags=cv2.INTER_LINEAR)

    histogram_image = getHistogram(warped_binary_image)

    left_fitx, right_fitx, ploty = fit_polynomial(warped_binary_image)

    final_image = render_lane(img, Minverse, ploty, left_fitx, right_fitx)
    
    return final_image

print ("calibrating camera")
ret, mtx, dist = calibrateCamera(9, 6)
print ("calibration done")

test = 0
if (test == 0):
    white_output = 'output_videos/solidWhiteRight.mp4'
    clip1 = VideoFileClip("project_video.mp4")
    white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
    %time white_clip.write_videofile(white_output, audio=False)
else:
    image_name = "straight_lines1.jpg"
    img = cv2.imread("test_images/" + image_name)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    final_image = process_image(img)
    #plot_images(img, main_binary_image)
    #plot_images(img, warped_binary_image)
    plot_images(img, final_image)


calibrating camera
calibration done
[MoviePy] >>>> Building video output_videos/solidWhiteRight.mp4
[MoviePy] Writing video output_videos/solidWhiteRight.mp4


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 1260/1261 [02:38<00:00,  7.43it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output_videos/solidWhiteRight.mp4 

Wall time: 2min 39s
